# Análisis de los diferentes modelos

## 1. Importación de librerías y modulos

In [1]:
#%pip install -r requirements.txt

In [14]:
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import faithfulness, answer_relevancy, context_precision, context_recall
from langchain_community.document_loaders.pdf import PyPDFDirectoryLoader
from langchain_community.llms import Ollama
from langchain_chroma import Chroma
import tqdm
from module import *

In [15]:
emb_model = "baai_small"
llm_model = "llama3"

## 2. Carga de datos 

In [16]:
documents_loader = PyPDFDirectoryLoader("./data/test3")
documents = documents_loader.load()

In [17]:
chunks = split_documents(documents)

## 3. Creación de la base de datos vectorial

In [18]:
# Load the existing database.
db = Chroma(
    persist_directory="./database_testing",
    embedding_function=get_embedding_function(emb_model)
)

# Calculate Page IDs.
chunks_with_ids = calculate_chunk_ids(chunks)

# Add or Update the documents.
existing_items = db.get(include=[])  # IDs are always included by default
existing_ids = set(existing_items["ids"])
print(f"Number of existing documents in DB: {len(existing_ids)}")

# Only add documents that don't exist in the DB.
new_chunks = []
for chunk in chunks_with_ids:
    if chunk.metadata["id"] not in existing_ids:
        new_chunks.append(chunk)

if len(new_chunks):
    print(f"👉 Adding new documents: {len(new_chunks)}")
    with tqdm.tqdm(total=len(new_chunks)) as pbar:
        for chunk in new_chunks:
            db.add_documents([chunk], ids=[chunk.metadata["id"]])
            pbar.update(1)
    print("Documents added correctly ✅")
else:
    print("✅ No new documents to add")

Number of existing documents in DB: 87
✅ No new documents to add


## 4. Preguntas de interés

Formulamos las preguntas y buscamos en la base de datos los chunks que nos ofrezcan mejor contexto para responderlas.

In [19]:
questions_docs_cat = [
    "Quins són els principals medicaments que poden desencadenar o exacerbar la insuficiència cardíaca segons l'article?",
    "Quins mecanismes estan implicats en la toxicitat miocardíaca induïda per certs fàrmacs?",
    "Com es categoritzen els medicaments segons el risc de provocar insuficiència cardíaca en pacients amb aquesta patologia?",
    "Com actuen els inhibidors de l'enzim convertidor d'angiotensina (IECA) en el tractament de la insuficiència cardíaca?",
    "Per què és important no interrompre el tractament farmacològic de la insuficiència cardíaca fins i tot si els símptomes milloren?",
    "Com es realitza el seguiment mèdic d'un pacient amb insuficiència cardíaca en tractament farmacològic?"
]

questions_docs_en = [
    "What is the fundamental postulate of Einstein's theory of special relativity?",
    "Explain the mass-energy equivalence formula and its significance in special relativity.",
    "How does special relativity reconcile with the principle of the constancy of the speed of light in a vacuum?",
    "Explain the concept of the 'Imitation Game' (Turing Test) and its significance in evaluating machine intelligence.",
    "What role does the concept of discrete state machines play in Turing's argument about machine intelligence?",
    "Discuss the significance of the concept of 'universal machines' as introduced by Turing in the context of machine intelligence."
]

questions_docs_es = [
    "¿Qué papel juega la arquitectura von Neumann en la estructura funcional de un ordenador?",
    "Explica la función de la Unidad de Control (UC) en un CPU y cómo afecta el rendimiento del ordenador.",
    "¿Cuál es la fórmula que define la ejecución de un programa en un ordenador según las instrucciones y datos de entrada?",
    "¿Qué es el algoritmo K-means y cómo se utiliza en el contexto del análisis de consumo energético?",
    "¿Cómo se determina el número óptimo de clústeres en un análisis con K-means?",
    "¿Qué metodología se utiliza para tratar los datos atípicos antes de aplicar el algoritmo K-means?"
]

questions = questions_docs_cat + questions_docs_en + questions_docs_es

In [20]:
contexts = []

for question in questions:
    # Get the top 5 most relevant documents
    results = db.similarity_search_with_score(question, k=5)

    # Make a list of the contexts
    question_contexts = []
    for doc, _score in results:
        question_contexts.append(doc.page_content)

    # Append the context sub-list to the list of contexts
    contexts.append(question_contexts)

print(contexts)

[['Si  se reunen los requisitos establecidos en la Ley 44/2007 de 13 de diciembre ( BOE 14 de diciembre ), la empresa se bonificará en la cuota\nempresarial a la Seguridad social en 850 euros/año ó la parte proporcional si el contrato es a tiempo partcial, durante tres años.\n      Si el contrato se suscribe con personas menores de 30 años, la bonificación será de 1650 euros/año durante tres años (Ley 11/2013).CON BONIFICACIÓN\n En lo no previsto en este   contrato, se estará a la legislación vigente que resulte de aplicación, y en particular , a lo disp uesto en el Estatuto\nde los Trabajadores, aprobado por el Real Decreto Legislativo 2/2015, de 23 de octubre (BOE. de 24 de octubre), y en la Ley 44/2007 ,  de 13 de\ndiciembre (BOE 14 de diciembre)y en lo dispuesto en la Sección I del Capítulo I de la ley 43/2006.  Asimismo le será de aplicaci ón lo dispuesto', 'CLÁUSULAS ESPECÍFICAS A TIEMPO PARCIAL CON VINCULACIÓN FORMATIVA\nTIEMPO P ARCIALCÓDIGO DE CONTRA TOMINISTERIO\nDE EMPLEO Y\

Ahora creamos las respuestas esperadas para cada pregunta.

In [21]:
answers_cat = [
    "Els principals medicaments que poden desencadenar o exacerbar la insuficiència cardíaca inclouen antiinflamatoris no esteroïdals (AINE), certs anestèsics, antiarrítmics, i alguns antidiabètics, entre d'altres, segons els mecanismes de toxicitat miocardíaca o disfunció cardíaca.",
    "Els mecanismes implicats en la toxicitat miocardíaca inclouen l'estrès oxidatiu, la inhibició de prostaglandines, la retenció de sodi i aigua, i la depressió de la funció miocàrdica, entre d'altres. Aquests mecanismes poden variar segons el fàrmac i la seva acció específica sobre el cor.",
    "Els fàrmacs associats amb la prolongació de l'interval QT i el risc de torsades de pointes inclouen amiodarona, citalopram, escitalopram, metadona, sotalol, entre d'altres. Aquests medicaments poden provocar arítmies greus, especialment en pacients amb insuficiència cardíaca.",
    "Els inhibidors de l'enzim convertidor d'angiotensina (IECA) actuen dilatant els vasos sanguinis, cosa que facilita el bombeig de sang per part del cor i redueix la pressió arterial, ajudant a millorar els símptomes de la insuficiència cardíaca.",
    "És important no interrompre el tractament perquè els medicaments no només milloren els símptomes, sinó que també prevenen l'empitjorament de la insuficiència cardíaca. Encara que el pacient es trobi bé, ha de continuar amb el tractament prescrit.",
    "El seguiment mèdic inclou la monitorització de la pressió arterial, el ritme cardíac, el pes del pacient, i la realització d'analítiques de sang per controlar els nivells de potassi, sodi, i la funció renal. També es poden fer electrocardiogrames per avaluar la funció cardíaca."
]

answers_en = [
    "The fundamental postulate of Einstein's theory of special relativity is that the laws of physics are the same in all inertial frames of reference, and that the speed of light in a vacuum is constant for all observers, regardless of the motion of the light source or the observer.",
    "The mass-energy equivalence formula is given by: E=mc^2, where E is the energy, m is the mass, and c is the speed of light. This formula signifies that mass and energy are interchangeable, and a small amount of mass can be converted into a large amount of energy.",
    "Special relativity reconciles with the principle of the constancy of the speed of light by postulating that the speed of light in a vacuum is the same for all observers, regardless of their motion relative to the light source. This leads to the need for a new understanding of space and time as being interwoven into a four-dimensional spacetime.",
    "The 'Imitation Game' involves a human interrogator communicating with both a human and a machine through written text, without knowing which is which. The interrogator's task is to determine which participant is the machine. If the machine can consistently convince the interrogator that it is human, it is considered to have passed the test, suggesting a form of intelligence.",
    "Discrete state machines are central to Turing's argument as they represent the basic structure of digital computers, which operate by transitioning between distinct states based on inputs. Turing argues that digital computers, as discrete state machines, can be programmed to simulate any other discrete state machine, including those that mimic human intelligence.",
    "The concept of 'universal machines' is significant because it implies that a single machine, with appropriate programming, can perform the tasks of any other machine. Turing suggests that this universality allows digital computers to potentially exhibit behaviors that we would classify as intelligent, supporting his thesis that machines can think."
]

answers_es = [
    "La arquitectura von Neumann es clave en la estructura funcional de un ordenador porque establece que tanto los datos como las instrucciones del programa se almacenan en la memoria principal, permitiendo que el ordenador ejecute programas de manera secuencial y eficiente.",
    "La Unidad de Control (UC) en un CPU es responsable de descodificar y ejecutar las instrucciones almacenadas en la memoria principal, enviando señales de control a las demás unidades del ordenador. La frecuencia del reloj de la UC, medida en MHz o GHz, determina en parte la velocidad de funcionamiento del ordenador, afectando su rendimiento.",
    "La fórmula que define la ejecución de un programa en un ordenador es: Datos de salida=f(Datos de entrada,Instrucciones), esta fórmula indica que los datos de salida de un programa dependen tanto de los datos de entrada como de las instrucciones del programa almacenado en el ordenador.",
    "El algoritmo K-means es un método de agrupamiento no jerárquico que particiona un conjunto de datos en un número específico de grupos (clústeres) basándose en las características compartidas. En el contexto del análisis de consumo energético, K-means se utiliza para identificar patrones de consumo similares entre diferentes clientes, permitiendo una mejor gestión y optimización de recursos energéticos.",
    "El número óptimo de clústeres en un análisis con K-means se determina mediante la evaluación de la precisión y la calidad de los clústeres utilizando medidas proporcionadas por la teoría de los conjuntos aproximados (RST), así como mediante la ejecución repetida del algoritmo con diferentes particiones iniciales.",
    "La metodología utilizada para tratar los datos atípicos antes de aplicar el algoritmo K-means incluye la detección de atípicos utilizando un rango de tres desviaciones estándar y la imputación de valores ausentes o atípicos mediante el valor medio del resto de las lecturas válidas, asegurando que los datos sean representativos y minimizando la distorsión en los clústeres resultantes."
]

answers = answers_cat + answers_en + answers_es
answers

['In the Arras Contract, the clause that specifies the conditions for the rescission of the contract is the third clause',
 'The worker will provide services as [job title], included in the professional group of [professional group], for the performance of [job functions] in accordance with the professional classification system in force in the company',
 'The buyer may terminate this contract, agreeing to forfeit the amounts paid as penalty deposits.',
 'En el Contrato de Arras, la cláusula que especifica las condiciones para la rescisión del contrato es la tercera cláusula',
 'El/la trabajador/a prestará sus servicios como [puesto], incluido en el grupo profesional de [grupo profesional], para la realización de las funciones de [funciones laborales] de acuerdo con el sistema de clasificación profesional vigente en la empresa',
 'La parte compradora podrá resolver el presente contrato, aviniéndose a perder las cantidades que ha entregado como arras penitenciales.',
 "En el Contracte d

Finalmente creamos el DataFrame para evaluar los modelos.

In [22]:
data_samples = {
    'question': questions,
    'answer': answers,
    'contexts': contexts,
    'ground_truth': answers
}

dataset = Dataset.from_dict(data_samples)

## 5. Evaluación de los modelos

In [23]:
embeddings = get_embedding_function(emb_model)
llm = Ollama(model=llm_model)

In [24]:
import nest_asyncio
from ragas.run_config import RunConfig


nest_asyncio.apply()

try:
    score = evaluate(
        dataset,
        metrics=[faithfulness, answer_relevancy],
        llm=llm,
        embeddings=embeddings,
        raise_exceptions=False,
        run_config=RunConfig(
            max_retries=30, # Default is 10
            max_wait=180, # Default is 60
            max_workers=64 # Default is 16
        )
    )
except Exception as e:
    print(f"An error ocurred: {e}")

df_score = score.to_pandas()
print(df_score)

Evaluating:   0%|          | 0/18 [00:00<?, ?it/s]

Failed to parse output. Returning None.
Failed to parse output. Returning None.
Failed to parse output. Returning None.
Failed to parse output. Returning None.
Failed to parse output. Returning None.
Failed to parse output. Returning None.
Failed to parse output. Returning None.
Failed to parse output. Returning None.


                                            question  \
0  What is the clause that specifies the conditio...   
1  What are the job functions described in the In...   
2  What penalties are applied if the buyer does n...   
3  ¿Cuál es la cláusula que especifica las condic...   
4  ¿Cuáles son las funciones laborales descritas ...   
5  ¿Qué sanciones se aplican si la parte comprado...   
6  Quina és la clàusula que especifica les condic...   
7  Quines són les funcions laborals descrites en ...   
8  Quines sancions s'apliquen si la part comprado...   

                                              answer  \
0  In the Arras Contract, the clause that specifi...   
1  The worker will provide services as [job title...   
2  The buyer may terminate this contract, agreein...   
3  En el Contrato de Arras, la cláusula que espec...   
4  El/la trabajador/a prestará sus servicios como...   
5  La parte compradora podrá resolver el presente...   
6  En el Contracte d'Arres, la clàusula que esp

In [25]:
df_score.head(10)

,question,answer,contexts,ground_truth,faithfulness,answer_relevancy
0,What is the clause that specifies the conditio...,"In the Arras Contract, the clause that specifi...",[Si se reunen los requisitos establecidos en ...,"In the Arras Contract, the clause that specifi...",NaN,1.000000
1,What are the job functions described in the In...,The worker will provide services as [job title...,[El trabajador es perceptor de prestaciones po...,The worker will provide services as [job title...,NaN,0.702688
2,What penalties are applied if the buyer does n...,"The buyer may terminate this contract, agreein...","[manifestación primera y, siendo a su vez el i...","The buyer may terminate this contract, agreein...",NaN,0.817590
3,¿Cuál es la cláusula que especifica las condic...,"En el Contrato de Arras, la cláusula que espec...",[CLÁUSULAS ESPECÍFICAS DE UN JÓVEN POR MICROE...,"En el Contrato de Arras, la cláusula que espec...",0.25,0.700195
4,¿Cuáles son las funciones laborales descritas ...,El/la trabajador/a prestará sus servicios como...,[El trabajador es perceptor de prestaciones po...,El/la trabajador/a prestará sus servicios como...,0.00,0.605152
5,¿Qué sanciones se aplican si la parte comprado...,La parte compradora podrá resolver el presente...,[Si se reunen los requisitos establecidos en ...,La parte compradora podrá resolver el presente...,NaN,0.000000
6,Quina és la clàusula que especifica les condic...,"En el Contracte d'Arres, la clàusula que espec...",[CLÁUSULAS ESPECÍFICAS DE UN JÓVEN POR MICROE...,"En el Contracte d'Arres, la clàusula que espec...",NaN,NaN
7,Quines són les funcions laborals descrites en ...,El/la treballador/a prestarà els seus serveis ...,[Se establece un período de adaptación al trab...,El/la treballador/a prestarà els seus serveis ...,0.00,0.663363
8,Quines sancions s'apliquen si la part comprado...,La part compradora podrà resoldre el present c...,"[manifestación primera y, siendo a su vez el i...",La part compradora podrà resoldre el present c...,0.00,0.650901
